<a href="https://colab.research.google.com/github/shkim0116/Lecture-Recommendation-NLP/blob/main/preprocess_w2v.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from tqdm import tqdm

data = pd.read_csv('/content/drive/Shareddrives/NLP모델링/코드 연습/문서/reviewtokenized.csv')
data['강의평토큰화'] = [list(map(str, data['강의평토큰화'].iloc[i][2:-2].split("', '"))) for i in range(len(data))]
data['target'] = [list(map(float, data['target'].iloc[i][1:-1].split(', '))) for i in range(len(data))]

In [ ]:
review = list(data['강의평토큰화'])
corpus = []
for i in review:
    a = ''
    for j in i:
        a += j + ' '
    corpus.append(a[:-1])

In [ ]:
tfidfv = TfidfVectorizer(min_df = 5).fit(corpus)
k = tfidfv.transform(corpus).toarray()

In [ ]:
new = []
for i in review:
    ddd = []
    for j in i:
        if len(j) > 1:ddd.append(j)
    new.append(ddd)

In [ ]:
model = Word2Vec(sentences = new, size=100, window = 5, min_count = 5, workers = 4, sg = 1)

In [ ]:
i = 1
for a in tfidfv.vocabulary_.keys():
    if i == 1:
        vector = model.wv[a][np.newaxis, :]
    else:
        vector = np.concatenate([vector, model.wv[a][np.newaxis, :]], axis = 0)
    i += 1

In [ ]:
sentence_vector = k @ vector
data['input'] = [list(sentence_vector[i]) for i in range(sentence_vector.shape[0])]

In [ ]:
uniquecla = list(set(data['강의명/교수명'].values))

In [ ]:
aggword = []
for j in tqdm(uniquecla):
    index = data[data['강의명/교수명']==j].index
    word = []
    for i in index:
        word += data['강의평토큰화'][i]
    aggword.append(word.copy())

100%|██████████| 386/386 [00:01<00:00, 358.83it/s]


In [ ]:
target = []
for i in uniquecla:
    target.append(data['target'][data['강의명/교수명']==i].values[0])

In [ ]:
data1 = pd.DataFrame({'강의명/교수명' : uniquecla,\
                      'target' : target,\
                      '강의평' : aggword})

In [ ]:
review1 = list(data1['강의평'])

corpus1 = []
for i in review1:
    a = ''
    for j in i:
        a += j + ' '
    corpus1.append(a[:-1])

In [ ]:
tfidfv1 = TfidfVectorizer(min_df = 5).fit(corpus1)
k1 = tfidfv1.transform(corpus1).toarray()

In [ ]:
i = 1
for a in tfidfv1.vocabulary_.keys():
    if i == 1:
        vector1 = model.wv[a][np.newaxis, :]
    else:
        vector1 = np.concatenate([vector1, model.wv[a][np.newaxis, :]], axis = 0)
    i += 1

In [ ]:
sentence_vector1 = k1 @ vector1
data1['input'] = [list(sentence_vector1[i]) for i in range(sentence_vector1.shape[0])]

In [ ]:
data1 = data1[['강의명/교수명','input','target']]

In [ ]:
data1.head(5)

,강의명/교수명,input,target
0,통계학입문/김숙경,"[-1.2990191696527762, -1.4830524926307118, 0.1...","[1.0, 0.5, 1.0, 1.0, 0.5, 0.5]"
1,글로벌리더십/김용호,"[-1.237644724973481, -1.4695318804965667, 0.07...","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]"
2,인문사회수학/김동호,"[-1.061686247201042, -1.268587524747092, -0.16...","[1.0, 0.5, 1.0, 0.5, 0.0, 0.5]"
3,스페인어(1)/최유정,"[-1.2145690852873476, -1.358813846804486, 0.20...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.5]"
4,"ROCK MUSIC, CULTURE AND SOCIETY/이지훈","[-1.2546117497950418, -1.387209685319639, 0.05...","[1.0, 0.0, 0.5, 1.0, 0.5, 0.5]"


In [ ]:
data2 = data.copy()
data2 = data2[['강의명/교수명', 'input', 'target']]

In [ ]:
aggvector = []
for j in tqdm(uniquecla):
    index = data2[data2['강의명/교수명']==j].index
    vector = np.zeros((100,))
    n = 0
    for i in index:
        vector += np.array(data2['input'][i])
        n += 1
    vector = vector / n
    aggvector.append(list(vector).copy())

100%|██████████| 386/386 [00:03<00:00, 110.31it/s]


In [ ]:
data3 = pd.DataFrame({'강의명/교수명' : uniquecla,\
                      'input' : aggvector,\
                      'target' : target})

In [ ]:
data3.head()

,강의명/교수명,input,target
0,통계학입문/김숙경,"[-0.31417282734761487, -0.3674974179630464, -0...","[1.0, 0.5, 1.0, 1.0, 0.5, 0.5]"
1,글로벌리더십/김용호,"[-0.3530267322174085, -0.3057673649840293, -0....","[0.5, 0.5, 0.5, 0.5, 0.5, 0.5]"
2,인문사회수학/김동호,"[-0.4518579473536773, -0.4531542860712264, -0....","[1.0, 0.5, 1.0, 0.5, 0.0, 0.5]"
3,스페인어(1)/최유정,"[-0.45684682930194104, -0.37327360611225824, -...","[1.0, 1.0, 1.0, 1.0, 0.0, 0.5]"
4,"ROCK MUSIC, CULTURE AND SOCIETY/이지훈","[-0.40634037789608635, -0.3194865621501923, -0...","[1.0, 0.0, 0.5, 1.0, 0.5, 0.5]"


In [ ]:
# data1.to_csv('/content/drive/Shareddrives/NLP모델링/코드 연습/문서/w2vaggver.csv')
# data3.to_csv('/content/drive/Shareddrives/NLP모델링/코드 연습/문서/w2vnormver.csv')